#### Weather

##### - import set up

In [1]:
# import .env keys
from dotenv import load_dotenv
load_dotenv()

import os
weather_api = os.environ['WEATHER_API_KEY']
import pandas as pd
import requests
import json
import sqlalchemy as sa
# get connection
from gans_connection import get_database_connection

##### - Create a function do get data from Airlabs API

In [3]:
def fetch_weather_data(city_id, latitude, longitude):
    try:
        api_url = f'http://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&appid={weather_api}&units=metric'
        response = requests.get(api_url)
        if response.status_code == 200:
            data = response.json()
            return data
    except Exception as e:
        print(f"Error {city_id} weather X.X =  {e}")
        return None

In [4]:
def weather():
    try:
        engine = get_database_connection()
        cities_table = pd.read_sql_table('cities', engine)
        weather_data = []

        for index, row in cities_table.iterrows():
            city_id = row['city_id']
            latitude = row['latitude']
            longitude = row['longitude']

            wthr_data = fetch_weather_data(city_id, latitude, longitude)
                
            if wthr_data is not None:
            # Extract relevant info every 3 hours
                for forecast in wthr_data['list']:
                # make sure if is not raining or snowing doest show an error
                    rain = forecast.get("rain", {}).get("3h", 0)
                    snow = forecast.get("snow", {}).get("3h", 0)
                
                    weather_data.append({
                        "city_id": city_id,
                        "temperature": forecast["main"]["temp"],
                        "feels_like": forecast["main"]["feels_like"],
                        "forecast": forecast["weather"][0]["main"],
                        "forecast_desc": forecast["weather"][0]["description"],
                        "wind_speed": forecast["wind"]["speed"],
                        "humidity": forecast["main"]["humidity"],
                        "rain": rain,
                        "snow": snow,
                        "forecast_ts": forecast["dt_txt"]
                    })

        weather_df = pd.DataFrame(weather_data)
        weather_df.to_sql('weather', engine, if_exists='append', index=False)
        return print("Sun is shining :D ")
    
    except Exception as e:
        print(f"Uh-oh, unexpected WEATHER anomaly X.X {e}")
        
    

In [5]:
weather()

Sun is shining :D 
